In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
from tqdm import tqdm
import statsmodels.api as sm

In [34]:
# 엑셀 파일을 읽어오는 코드 수정
# 9번째 행(파이썬 인덱스로는 8)을 칼럼 이름으로 사용하고, 이전 행들은 무시합니다.
df_0 = pd.read_excel('quant_mid.xlsx', header=8)

# 데이터프레임의 첫번째 열을 'Date'로 이름을 변경하고 인덱스로 설정
df_0.rename(columns={df_0.columns[0]: 'Date'}, inplace=True)
df_0.set_index('Date', inplace=True)

# 결과 확인
df_0.head()

# 'Symbol Name' 행을 칼럼 이름으로 사용하고, 필요한 데이터만 포함하는 새 데이터프레임 생성
df_filtered = df_0.iloc[5:]  # 'Frequency' 행 이후의 데이터만 포함
df_filtered.columns = df_0.loc['Symbol Name']  # 'Symbol Name' 행의 값으로 칼럼 이름 재설정

# 결과 확인
df_filtered.head()

# df_filtered.to_excel('example.xlsx')


Symbol Name,삼성전자,SK하이닉스,LG에너지솔루션,삼성바이오로직스,현대차,기아,셀트리온,POSCO홀딩스,삼성SDI,LG화학,...,에스앤씨엔진그룹,차이나하오란,웨이포트,성융광전투자,완리,평산차업 KDR,네프로아이티,중국고섬,SBI모기지,SNK
Date,,,,,,,,,,,,,,,,,,,,,
1990-07-31 00:00:00,358,NaN,NaN,NaN,11960,137477,NaN,21771,15857,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-08-31 00:00:00,303,NaN,NaN,NaN,10899,120293,NaN,20048,14101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-09-30 00:00:00,311,NaN,NaN,NaN,11312,129666,NaN,20411,14243,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-10-31 00:00:00,349,NaN,NaN,NaN,13256,141383,NaN,21046,16095,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-11-30 00:00:00,331,NaN,NaN,NaN,11901,137477,NaN,20683,15667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_2 = pd.read_excel('example.xlsx')
df_2.set_index('Date', inplace=True)
df_2.head()

,삼성전자,SK하이닉스,LG에너지솔루션,삼성바이오로직스,현대차,기아,셀트리온,POSCO홀딩스,삼성SDI,LG화학,...,에스앤씨엔진그룹,차이나하오란,웨이포트,성융광전투자,완리,평산차업 KDR,네프로아이티,중국고섬,SBI모기지,SNK
Date,,,,,,,,,,,,,,,,,,,,,
1990-07-31,358,NaN,NaN,NaN,11960,137477,NaN,21771,15857,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-08-31,303,NaN,NaN,NaN,10899,120293,NaN,20048,14101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-09-30,311,NaN,NaN,NaN,11312,129666,NaN,20411,14243,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-10-31,349,NaN,NaN,NaN,13256,141383,NaN,21046,16095,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-11-30,331,NaN,NaN,NaN,11901,137477,NaN,20683,15667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df = df_2.copy()

# 상장폐지에 대한 처리

for column in df.columns:
    nan_start_index = None  # NaN 시작 인덱스 초기화
    for i in range(1, len(df)):
        if df[column].iloc[i-1] > 0 and pd.isna(df[column].iloc[i]):
            if nan_start_index is None:  # 연속적인 NaN 중 첫 번째 위치 저장
                nan_start_index = i
        else:
            if nan_start_index is not None:
                df.at[df.index[nan_start_index], column] = 0
                nan_start_index = None  # 리셋

    # 연속되는 마지막 NaN 체크
    if nan_start_index is not None:
        df.at[df.index[nan_start_index], column] = 0

print(df)

             삼성전자    SK하이닉스  LG에너지솔루션  삼성바이오로직스     현대차      기아      셀트리온  \
Date                                                                        
1990-07-31    358       NaN       NaN       NaN   11960  137477       NaN   
1990-08-31    303       NaN       NaN       NaN   10899  120293       NaN   
1990-09-30    311       NaN       NaN       NaN   11312  129666       NaN   
1990-10-31    349       NaN       NaN       NaN   13256  141383       NaN   
1990-11-30    331       NaN       NaN       NaN   11901  137477       NaN   
...           ...       ...       ...       ...     ...     ...       ...   
2023-08-31  66900  121800.0  544000.0  737000.0  189100   80200  143900.0   
2023-09-30  68400  114700.0  476500.0  681000.0  191100   81400  139200.0   
2023-10-31  66900  116300.0  385500.0  709000.0  169700   76900  149500.0   
2023-11-30  72800  133900.0  454500.0  725000.0  184300   85900  162900.0   
2023-12-31  78500  141500.0  427500.0  760000.0  203500  100000  201500.0   

# 문제1

In [37]:
# df = df_2.copy()

# 모멘텀 및 수익률 계산 함수 정의
def calculate_momentum_and_return(df, J, K):
    # 롤링 윈도우로 J개월 모멘텀 계산
    momentum = df.pct_change(periods=J).shift(-J)
    
    # 롤링 윈도우로 K개월 후 수익률 계산
    future_return = df.shift(-J-K) / df.shift(-J) - 1
    
    # J 기간 내 NaN을 포함하고, K 기간 후에 NaN을 포함하는 행 제거
    momentum = momentum[~(df.rolling(window=J).apply(lambda x: x.isnull().any()).astype(bool))]
    future_return = future_return[~(df.shift(-J).rolling(window=K).apply(lambda x: x.isnull().any()).astype(bool))]
    
    # 상위 30%와 하위 30%의 종목 식별
    top_30_threshold = momentum.quantile(0.7, axis=1)
    bottom_30_threshold = momentum.quantile(0.3, axis=1)
    buy_signals = momentum.gt(top_30_threshold, axis=0)
    sell_signals = momentum.lt(bottom_30_threshold, axis=0)
    
    # 신호에 따른 수익률 계산
    buy_returns = future_return[buy_signals].mean(axis=1)
    sell_returns = future_return[sell_signals].mean(axis=1)
    buy_sell_returns = buy_returns - sell_returns
    
    return buy_returns.mean(), sell_returns.mean(), buy_sell_returns.mean()

# J와 K의 값들을 정의
J_values = [3, 6,9,12]  # 예시 데이터로 J 값을 변경
K_values = [3, 6,9,12]  # 예시 데이터로 K 값을 변경

# 결과를 저장할 데이터프레임을 초기화
results = pd.DataFrame(index=J_values, columns=K_values)

# 모든 J와 K 조합에 대해 계산 수행
for J in J_values:
    for K in K_values:
        buy_avg, sell_avg, buy_sell_avg = calculate_momentum_and_return(df, J, K)
        # 결과 저장
        results.at[J, K] = {
            'Buy': buy_avg,
            'Sell': sell_avg,
            'Buy-sell': buy_sell_avg
        }

# 결과 출력
results_stacked = results.stack().reset_index()
results_stacked.columns = ['J', 'K', 'Data']
results_pivot = results_stacked.pivot(index='J', columns='K', values='Data')

# 결과 출력 (포맷 변경 포함)
results_formatted = results_pivot.applymap(lambda x: f"{x['Buy']:.4f}, {x['Sell']:.4f}, {x['Buy-sell']:.4f}" if isinstance(x, dict) else np.nan)
results_formatted


K,3,6,9,12
J,,,,
3,"0.0382, 0.0407, -0.0025","0.0763, 0.0876, -0.0113","0.1207, 0.1398, -0.0190","0.1543, 0.2045, -0.0502"
6,"0.0363, 0.0425, -0.0062","0.0777, 0.0863, -0.0086","0.1106, 0.1421, -0.0316","0.1313, 0.2291, -0.0978"
9,"0.0384, 0.0406, -0.0022","0.0734, 0.0845, -0.0111","0.0953, 0.1516, -0.0563","0.1140, 0.2344, -0.1204"
12,"0.0339, 0.0436, -0.0096","0.0604, 0.0984, -0.0380","0.0805, 0.1605, -0.0799","0.1026, 0.2336, -0.1310"


In [33]:


# Define J and K values
J_values = [3, 6, 9, 12]
K_values = [3, 6, 9, 12]

# Initialize the results DataFrame
results = pd.DataFrame(index=pd.MultiIndex.from_product([J_values, ['Sell', 'Buy', 'Buy-Sell']]), columns=K_values)

# Function to calculate momentum strategy
def calculate_momentum_strategy(df, J, K):
    """
    This function calculates the momentum strategy based on J months of returns and holds for K months.
    It returns the mean returns for buy, sell, and buy-sell portfolios.
    """
    # Rolling window to calculate past J months' momentum
    momentum = df.pct_change(J).shift(-J)
    
    # Rank stocks based on momentum
    ranks = momentum.rank(axis=1, method='first', ascending=False)
    
    # Define thresholds for selection
    buy_threshold = ranks.quantile(0.7, axis=1, interpolation='higher')
    sell_threshold = ranks.quantile(0.3, axis=1, interpolation='higher')

    buy_signals = ranks.le(buy_threshold, axis=0)
    sell_signals = ranks.ge(sell_threshold, axis=0)
    
    # Calculate future returns for K months
    future_returns = df.shift(-K).pct_change(K)

    # Buy, Sell portfolio returns
    buy_returns = future_returns[buy_signals].mean(axis=1)
    sell_returns = future_returns[sell_signals].mean(axis=1)
    
    # Calculate Buy-Sell returns
    buy_sell_returns = buy_returns - sell_returns

    # Replace NaN with -1 for delisted stocks
    buy_sell_returns = buy_sell_returns.fillna(-1)

    return buy_returns.mean(), sell_returns.mean(), buy_sell_returns.mean()

# Iterate through all J and K combinations
for J in J_values:
    for K in K_values:
        buy_avg, sell_avg, buy_sell_avg = calculate_momentum_strategy(df, J, K)
        results.at[(J, 'Buy'), K] = buy_avg
        results.at[(J, 'Sell'), K] = sell_avg
        results.at[(J, 'Buy-Sell'), K] = buy_sell_avg

# Convert the multi-level index into two columns
results.reset_index(inplace=True)
results.columns = ['J', 'Strategy', '3', '6', '9', '12']

# Filter the strategies for display based on the uploaded image structure
buy_results = results[results['Strategy'] == 'Buy'].drop('Strategy', axis=1).set_index('J')
sell_results = results[results['Strategy'] == 'Sell'].drop('Strategy', axis=1).set_index('J')
buy_sell_results = results[results['Strategy'] == 'Buy-Sell'].drop('Strategy', axis=1).set_index('J')

# Combine buy, sell, and buy-sell results
combined_results = pd.concat([sell_results, buy_results, buy_sell_results], keys=['Sell', 'Buy', 'Buy-Sell'])

# Display the results
combined_results


3         6         9        12
         J                                         
Sell     3  -0.109238 -0.076735 -0.041188 -0.007435
         6  -0.069946 -0.146723 -0.113056 -0.080495
         9  -0.051398 -0.108544 -0.171787 -0.138802
         12 -0.040743 -0.086439 -0.135552 -0.191022
Buy      3        inf       inf       inf       inf
         6        inf       inf       inf       inf
         9        inf       inf       inf       inf
         12       inf       inf       inf       inf
Buy-Sell 3        inf       inf       inf       inf
         6        inf       inf       inf       inf
         9        inf       inf       inf       inf
         12       inf       inf       inf       inf

In [38]:

import numpy as np

# 모멘텀 및 수익률 계산 함수 정의
def calculate_momentum_and_return(df, J, K):
    # 롤링 윈도우로 J개월 모멘텀 계산
    momentum = df.pct_change(periods=J).shift(-J)
    
    # 롤링 윈도우로 K개월 후 수익률 계산
    future_return = df.shift(-J-K) / df.shift(-J) - 1
    
    # J 기간 내 NaN을 포함하는 종목 제거, K 기간에 NaN값을 0으로 치환
    window_data = df.rolling(window=J).apply(lambda x: x.dropna().mean() if x.dropna().count() > J/2 else np.nan)
    future_window = df.shift(-J-K).rolling(window=K).apply(lambda x: x.fillna(0).mean())
    
    # 롤링 윈도우 내에 NaN 값을 포함하는 행 제거
    momentum = momentum[~window_data.isnull()]
    future_return = future_return[~future_window.isnull()]
    
    # 상위 30%와 하위 30%의 종목 식별
    top_30_threshold = momentum.quantile(0.7, axis=1)
    bottom_30_threshold = momentum.quantile(0.3, axis=1)
    buy_signals = momentum.gt(top_30_threshold, axis=0)
    sell_signals = momentum.lt(bottom_30_threshold, axis=0)
    
    # 신호에 따른 수익률 계산
    buy_returns = future_return[buy_signals].mean(axis=1)
    sell_returns = future_return[sell_signals].mean(axis=1)
    buy_sell_returns = buy_returns - sell_returns
    
    return buy_returns.mean(), sell_returns.mean(), buy_sell_returns.mean()

# J와 K의 값들을 정의
J_values = [3,6,9,12 ]  # 예시 데이터로 J 값을 변경
K_values = [3, 6,9,12]  # 예시 데이터로 K 값을 변경

# 결과를 저장할 데이터프레임을 초기화
results = pd.DataFrame(index=J_values, columns=K_values)

# 모든 J와 K 조합에 대해 계산 수행
for J in J_values:
    for K in K_values:
        buy_avg, sell_avg, buy_sell_avg = calculate_momentum_and_return(df, J, K)
        # 결과 저장
        results.at[J, K] = {
            'Buy': buy_avg,
            'Sell': sell_avg,
            'Buy-sell': buy_sell_avg
        }

# 결과 출력
results_stacked = results.stack().reset_index()
results_stacked.columns = ['J', 'K', 'Data']
results_pivot = results_stacked.pivot(index='J', columns='K', values='Data')

# 결과 출력 (포맷 변경 포함)
results_formatted = results_pivot.applymap(lambda x: f"{x['Buy']:.4f}, {x['Sell']:.4f}, {x['Buy-sell']:.4f}" if isinstance(x, dict) else np.nan)
results_formatted



K,3,6,9,12
J,,,,
3,"0.0382, 0.0407, -0.0025","0.0763, 0.0876, -0.0113","0.1214, 0.1394, -0.0180","0.1554, 0.2049, -0.0495"
6,"0.0363, 0.0425, -0.0062","0.0777, 0.0863, -0.0086","0.1106, 0.1421, -0.0316","0.1313, 0.2291, -0.0978"
9,"0.0384, 0.0406, -0.0022","0.0734, 0.0845, -0.0111","0.0953, 0.1516, -0.0563","0.1140, 0.2344, -0.1204"
12,"0.0339, 0.0436, -0.0096","0.0604, 0.0984, -0.0380","0.0805, 0.1605, -0.0799","0.1026, 0.2337, -0.1310"


In [72]:
# 결과 출력 (포맷 변경 포함)
results_formatted = results_pivot.applymap(lambda x: pd.Series({
    'Buy': f"{x['Buy']:.4f}",
    'Sell': f"{x['Sell']:.4f}",
    'Buy-sell': f"{x['Buy-sell']:.4f}"
}) if isinstance(x, dict) else np.nan)

# 표시 형식 변경
results_formatted = results_formatted.applymap(lambda x: '\n'.join([f"{index}: {value}" for index, value in x.items()]))
results_formatted


K,3,6,9,12
J,,,,
3,Buy: 0.0382\nSell: 0.0407\nBuy-sell: -0.0025,Buy: 0.0763\nSell: 0.0876\nBuy-sell: -0.0113,Buy: 0.1214\nSell: 0.1394\nBuy-sell: -0.0180,Buy: 0.1554\nSell: 0.2049\nBuy-sell: -0.0495
6,Buy: 0.0363\nSell: 0.0425\nBuy-sell: -0.0062,Buy: 0.0777\nSell: 0.0863\nBuy-sell: -0.0086,Buy: 0.1106\nSell: 0.1421\nBuy-sell: -0.0316,Buy: 0.1313\nSell: 0.2291\nBuy-sell: -0.0978
9,Buy: 0.0384\nSell: 0.0406\nBuy-sell: -0.0022,Buy: 0.0734\nSell: 0.0845\nBuy-sell: -0.0111,Buy: 0.0953\nSell: 0.1516\nBuy-sell: -0.0563,Buy: 0.1140\nSell: 0.2344\nBuy-sell: -0.1204
12,Buy: 0.0339\nSell: 0.0436\nBuy-sell: -0.0096,Buy: 0.0604\nSell: 0.0984\nBuy-sell: -0.0380,Buy: 0.0805\nSell: 0.1605\nBuy-sell: -0.0799,Buy: 0.1026\nSell: 0.2337\nBuy-sell: -0.1310


In [73]:
# 결과 출력 (포맷 변경 포함)
results_formatted = results_pivot.applymap(lambda x: pd.Series({
    'Buy': f"{x['Buy']:.4f}",
    'Sell': f"{x['Sell']:.4f}",
    'Buy-sell': f"{x['Buy-sell']:.4f}"
}) if isinstance(x, dict) else np.nan)

# 표시 형식 변경
results_formatted = results_formatted.applymap(lambda x: '\n'.join([f"{index}: {value}" for index, value in x.items()]))
results_formatted

K,3,6,9,12
J,,,,
3,Buy: 0.0382\nSell: 0.0407\nBuy-sell: -0.0025,Buy: 0.0763\nSell: 0.0876\nBuy-sell: -0.0113,Buy: 0.1214\nSell: 0.1394\nBuy-sell: -0.0180,Buy: 0.1554\nSell: 0.2049\nBuy-sell: -0.0495
6,Buy: 0.0363\nSell: 0.0425\nBuy-sell: -0.0062,Buy: 0.0777\nSell: 0.0863\nBuy-sell: -0.0086,Buy: 0.1106\nSell: 0.1421\nBuy-sell: -0.0316,Buy: 0.1313\nSell: 0.2291\nBuy-sell: -0.0978
9,Buy: 0.0384\nSell: 0.0406\nBuy-sell: -0.0022,Buy: 0.0734\nSell: 0.0845\nBuy-sell: -0.0111,Buy: 0.0953\nSell: 0.1516\nBuy-sell: -0.0563,Buy: 0.1140\nSell: 0.2344\nBuy-sell: -0.1204
12,Buy: 0.0339\nSell: 0.0436\nBuy-sell: -0.0096,Buy: 0.0604\nSell: 0.0984\nBuy-sell: -0.0380,Buy: 0.0805\nSell: 0.1605\nBuy-sell: -0.0799,Buy: 0.1026\nSell: 0.2337\nBuy-sell: -0.1310
